#### 使用Scipy求解SVM

In [1]:
from scipy.optimize import minimize
import numpy as np
class SVM:
    def __init__(self,C=0.001):
        self.c=C
        
    def fit(self,X,y):
        y=2*y-1
        m,n=X.shape
        unknow_vector=np.ones(m+n+1)/(m+n+1)
        
        self.solver=minimize(type(self).minimize_object,unknow_vector,args=(X,y,self.c),
              constraints=({'type': 'ineq', 'args': (X,y),
                            'fun':lambda unknow_vector,X,y:type(self).linear_onstraint(unknow_vector,X,y)},
                           {'type': 'ineq', 'args': (y,),'fun':lambda unknow_vector,y:type(self).relaxation_constraint(unknow_vector,y)}                         
             ))
        self.coef_=self.solver.x[:n]
        self.intercept_=b=self.solver.x[n]
        self.beta_=self.solver.x[n+1:]
        
        self.support_value=y*(np.dot(X,self.coef_.T)+self.intercept_)+self.beta_
        self.support_=np.where(self.support_value<=1.01)
        self.support_vectors_=X[self.support_]
           
        return self
    
    def predict(self,X):
        pred=X@self.coef_+self.intercept_
        return (np.sign(pred)+1)/2
        
    
    
    def minimize_object(unknow_vector,X,y,c):
        m,n=X.shape
        w=unknow_vector[:n]  
        b=unknow_vector[n]   
        beta=unknow_vector[n+1:]
        object_value=0.5*np.sum(w**2)+c*np.sum(beta)
        return object_value

    def linear_onstraint(unknow_vector,X,y):
        m,n=X.shape
        w=unknow_vector[:n]
        b=unknow_vector[n]
        beta=unknow_vector[n+1:]
        return y*(X@w.T+b)+beta-1

    def relaxation_constraint(unknow_vector,y):
        n=len(y)
        beta=unknow_vector[n+1:]    
        return beta

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X,y=load_breast_cancer().data,load_breast_cancer().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
my_svm=SVM(C=0.00001).fit(X_train,y_train)
pred=my_svm.predict(X_test)
pred

array([1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       1.])

In [3]:
accuracy_score(y_test,pred)

0.9122807017543859

In [4]:
my_svm.support_vectors_

array([[1.377e+01, 2.229e+01, 9.063e+01, 5.889e+02, 1.200e-01, 1.267e-01,
        1.385e-01, 6.526e-02, 1.834e-01, 6.877e-02, 6.191e-01, 2.112e+00,
        4.906e+00, 4.970e+01, 1.380e-02, 3.348e-02, 4.665e-02, 2.060e-02,
        2.689e-02, 4.306e-03, 1.639e+01, 3.401e+01, 1.116e+02, 8.069e+02,
        1.737e-01, 3.122e-01, 3.809e-01, 1.673e-01, 3.080e-01, 9.333e-02],
       [1.486e+01, 1.694e+01, 9.489e+01, 6.737e+02, 8.924e-02, 7.074e-02,
        3.346e-02, 2.877e-02, 1.573e-01, 5.703e-02, 3.028e-01, 6.683e-01,
        1.612e+00, 2.392e+01, 5.756e-03, 1.665e-02, 1.461e-02, 8.281e-03,
        1.551e-02, 2.168e-03, 1.631e+01, 2.054e+01, 1.023e+02, 7.775e+02,
        1.218e-01, 1.550e-01, 1.220e-01, 7.971e-02, 2.525e-01, 6.827e-02],
       [1.246e+01, 2.404e+01, 8.397e+01, 4.759e+02, 1.186e-01, 2.396e-01,
        2.273e-01, 8.543e-02, 2.030e-01, 8.243e-02, 2.976e-01, 1.599e+00,
        2.039e+00, 2.394e+01, 7.149e-03, 7.217e-02, 7.743e-02, 1.432e-02,
        1.789e-02, 1.008e-02, 1.509e